In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

*Importing Libraries*

In [ ]:
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt
import seaborn as sns

from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

# for data overview
from pandas_profiling import ProfileReport
import plotly.graph_objects as go
fig = go.Figure()
import re




*Importing CSV file*

In [ ]:
movie_data = pd.read_csv("/kaggle/input/movies-on-netflix-prime-video-hulu-and-disney/MoviesOnStreamingPlatforms_updated.csv")

*Dataset Overview*

In [ ]:
movie_data.head()

In [ ]:
movie_data.shape

In [ ]:
movie_data.info()

*dropping country and directors*

In [ ]:
df=pd.DataFrame(movie_data)
df.drop(['Unnamed: 0','Directors','Country','Type'], axis=1, inplace= True)
df.head()

*NULL Values*

In [ ]:
movie_data.isnull().sum()

*Dealing with NULL Values*

In [ ]:
Age_Null = round(movie_data['Age'].isnull().sum()/len(movie_data['Age']) * 100 , 2)
Genres_Null = round(movie_data['Genres'].isnull().sum()/len(movie_data['Genres']) * 100 , 2)
Directors_Null = round(movie_data['Directors'].isnull().sum()/len(movie_data['Directors']) * 100 , 2)
Runtime_Null = round(movie_data['Runtime'].isnull().sum()/len(movie_data['Runtime']) * 100 , 2)
Language_Null = round(movie_data['Language'].isnull().sum()/len(movie_data['Language']) * 100 , 2)
Country_Null = round(movie_data['Country'].isnull().sum()/len(movie_data['Country']) * 100 , 2)    
Rotten_Null = round(movie_data['Rotten Tomatoes'].isnull().sum()/len(movie_data['Rotten Tomatoes']) * 100 , 2)                          

In [ ]:
print("Age Null: {}%".format(Age_Null))
print("Genres_Null: {}%".format(Genres_Null))
print("Directors_Null: {}%".format(Directors_Null))
print("Runtime_Null : {}%".format(Runtime_Null))
print("Language_Null: {}%".format(Language_Null))
print("Country_Null: {}%".format(Country_Null))
print("Rotten_Null: {}%".format(Rotten_Null))

In [ ]:
#eg: let us deal with age 
movie_data['Age'].value_counts(), movie_data['Age'].shape

In [ ]:
movie_data['Age'].value_counts().iplot('bar')

*values missing age input*

In [ ]:
movie_data[movie_data['Age'].isnull()].shape

In [ ]:
movie_data[movie_data['Age'].isnull()]

*Exploratory Data Analysis*

In [ ]:
print(movie_data.columns)
movie_data.head(3)

1. Rotten tomatoes ratings

In [ ]:
movie_data['Rotten Tomatoes'].value_counts()

In [ ]:
#rounding off for histogram representation
import re

def convert_str_to_int(val):
    new_val =  re.sub('%','',val)
    return(int(new_val))

def round_fix(data):
    data_str = str(data).strip()
    if data_str != 'nan':
        data = convert_str_to_int(data_str)
        if data in range(0,11):
            return '10'
        if data in range(11,21):
            return '20'
        if data in range(21,31):
            return '30'
        if data in range(31,41):
            return '40'
        if data in range(41,51):
            return '50'
        if data in range(51,61):
            return '60'
        if data in range(61,71):
            return '70'
        if data in range(71,81):
            return '80'
        if data in range(81,91):
            return '90'

        if data in range(91,101):
            return '100'


movie_data['Rotten_Tomatoes_overview'] = movie_data['Rotten Tomatoes'].apply(round_fix)

In [ ]:
movie_data['Rotten_Tomatoes_overview'].value_counts().iplot(kind='bar', bins=20, xTitle = 'Rotten Tomatoes Ratings', yTitle='Number of movies', title='OTT ROtten TOmatoes')

*which platform has better collection*

In [ ]:
netflix_count = movie_data[movie_data['Rotten_Tomatoes_overview'] == '100']['Netflix'].sum()
Hulu_count = movie_data[movie_data['Rotten_Tomatoes_overview'] == '100']['Hulu'].sum()
Disney_count = movie_data[movie_data['Rotten_Tomatoes_overview'] == '100']['Disney+'].sum()
prime_count = movie_data[movie_data['Rotten_Tomatoes_overview'] == '100']['Prime Video'].sum()

indexes = ['Netflix', 'Hulu', 'Disney', 'Amazon Prime']
values = [netflix_count, Hulu_count, Disney_count,prime_count]
for x,y in zip(indexes,values):
    print(x,y)

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(
rows=1,cols=2, subplot_titles=["Highest Rottem Tomatoes movies"],
specs=[[{'type':'bar'},{'type':'pie'}]])

fig.add_trace(go.Bar(x=indexes, y=values), row=1,col=1)
fig.add_trace(go.Pie(labels=indexes, values=values), row=1,col=2)

*IMDB Ratings*

In [ ]:
movie_data['IMDb'].value_counts()

In [ ]:
#rounding off
def round_val(data):
    if str(data) != 'nan':
        return round(data)
    
movie_data['IMDB_group'] = movie_data['IMDb'].apply(round_val)
index = movie_data['IMDB_group'].value_counts().sort_index(ascending=False).index.tolist() #rating
values = movie_data['IMDB_group'].value_counts().sort_index(ascending=False).tolist() #amount
print(index,values)
for x,y in zip(index,values):
    print(x,y)

In [ ]:
import seaborn as sns
movie_data['IMDB_group'].value_counts().iplot('bar', xTitle='IMDB Rating', yTitle='Num of Movies', title='IMDB RATING OVERVIEW')

*imdb movies on diff platform*

In [ ]:
Netflix_count = movie_data[movie_data['IMDB_group'] == 9]['Netflix'].sum()
Hulu_count = movie_data[movie_data['IMDB_group'] == 9]['Hulu'].sum()
Disney_count = movie_data[movie_data['IMDB_group'] == 9]['Disney+'].sum()
prime_count = movie_data[movie_data['IMDB_group'] == 9]['Prime Video'].sum()

indexes = ['Netflix', 'Hulu', 'Disney', 'Amazon Prime']
values = [netflix_count, Hulu_count, Disney_count,prime_count]
for x,y in zip(indexes,values):
    print(x,y)

In [ ]:
fig = make_subplots(
rows=1,cols=2, subplot_titles=["Top IMDB Rated Movies"],
specs=[[{'type':'bar'},{'type':'pie'}]])

fig.add_trace(go.Bar(x=indexes, y=values), row=1,col=1)
fig.add_trace(go.Pie(labels=indexes, values=values), row=1,col=2)

*Top movies and platforms*

In [ ]:
prime_movies = movie_data[(movie_data['IMDB_group'] == 9) & (movie_data['Prime Video'] == 1)]
print(prime_movies.shape)

*Recommended Movies*

In [ ]:
def netflix_platform(data):
    if data == 1:
        return 'netflix'
    else:
        return data

def hulu_platform(data):
    if data == 1:
        return 'hulu'
    else:
        return data
    
def disney_platform(data):
    if data == 1:
        return 'disney'
    else:
        return data
    
def prime_platform(data):
    if data == 1:
        return 'prime'
    else:
        return data

    
movie_data['Netflix'] = movie_data['Netflix'].apply(netflix_platform)
movie_data['Hulu'] = movie_data['Hulu'].apply(hulu_platform)    
movie_data['Prime Video'] = movie_data['Prime Video'].apply(prime_platform)  
movie_data['Disney+'] = movie_data['Disney+'].apply(disney_platform)  

In [ ]:
movie_data

*Genres*

In [ ]:
def check_thriller(data):

    if str(data).strip() != 'nan':
        if 'horror' in data.lower():
            return 'horror'
        elif 'thriller' in data.lower():
            return 'thriller'
        elif 'sci-fi' in data.lower():
            return 'sci-fi'
        elif 'documentary' in data.lower():
            return 'documentary'
        elif 'action' in data.lower():
            return 'action'
        elif 'animation' in data.lower():
            return 'animation'
        elif 'comedy' in data.lower():
            return 'comedy'
        elif 'western' in data.lower():
            return  'western'
        elif 'drama' in data.lower():
            return 'drama'
        elif 'fantasy' in data.lower():
            return 'fantasy'
        elif 'romance' in data.lower():
            return 'romance'
        elif 'music' in data.lower():
            return 'music'
        elif 'adventure' in data.lower():
            return 'adventure'
        elif 'sport' in data.lower():
            return 'sport'
        elif 'reality-tv' in data.lower() or 'talk-show' in data.lower() or 'game-show' in data.lower():
            return 'tv-show'
        elif 'history' in data.lower():
            return 'history'
        elif 'family' in data.lower():
            return 'family'
        elif 'biography' in data.lower():
            return 'biography'
        elif 'biography' in data.lower():
            return 'biography'
        elif 'mystery' in data.lower():
            return 'Mystery'
        elif 'war' in data.lower():
            return 'war'
        
movie_data['mov_type'] = movie_data['Genres'].apply(check_thriller)

In [ ]:
movie_data['mov_type'].value_counts().iplot('bar')

*Movies on Online Platforms*

In [ ]:
 top_movie = movie_data[movie_data['IMDB_group'] == 9]

In [ ]:
net_index = top_movie[top_movie['Netflix']=='netflix']['mov_type'].value_counts().index.tolist()
net_val = (top_movie[top_movie['Netflix']=='netflix']['mov_type'].value_counts().values.tolist())

prime_index = top_movie[top_movie['Prime Video']=='prime']['mov_type'].value_counts().index.tolist()
prime_val = (top_movie[top_movie['Prime Video']=='prime']['mov_type'].value_counts().values.tolist())

disney_index = top_movie[top_movie['Disney+']=='disney']['mov_type'].value_counts().index.tolist()
disney_val = (top_movie[top_movie['Disney+']=='disney']['mov_type'].value_counts().values.tolist())

hulu_index = top_movie[top_movie['Hulu']=='hulu']['mov_type'].value_counts().index.tolist()
hulu_val = (top_movie[top_movie['Hulu']=='hulu']['mov_type'].value_counts().values.tolist())

In [ ]:
fig = make_subplots(
rows=1,cols=2, subplot_titles=["Netflix vs Amazon Prime"],
specs=[[{'type':'pie'},{'type':'pie'}]])

fig.add_trace(go.Pie(labels=net_index, values=net_val, title='Netflix'), row=1,col=1)
fig.add_trace(go.Pie(labels=prime_index, values=prime_val, title='Prime'), row=1,col=2)
fig.update_layout(height=800, width=1000, title_text='Top IMDB Movies/Show')


*User age graph*

In [ ]:
movie_data['Age'].value_counts().iplot('bar', xTitle='Age Group', yTitle='Movies',title='Age Data')

*Amazon Prime Overview*

In [ ]:
prime_movies = movie_data[movie_data['Prime Video'] == 'prime']

In [ ]:
prime_movies.drop(['Hulu','Disney+','Netflix', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
prime_movies['Language'].value_counts()[:30].iplot('bar')

In [ ]:
# Runtime
runtime_net = prime_movies.sort_values(by='Runtime',ascending=False).head(20)
runtime_net

sns.barplot(data=runtime_net, y='Title',x='Runtime')
plt.title('Top Runtime movies on Amazon Prime')

In [ ]:
net_movies = prime_movies[prime_movies['IMDb'] > 8]
net_movies_count = net_movies.shape[0]
print("Movies with IMDB 8+ Rating in Amazon Prime: {}".format(net_movies_count))

In [ ]:
net_movies = net_movies.sort_values(by='IMDb', ascending=False).head(10)
sns.barplot(data=net_movies, y='Title',x='IMDb')
plt.title('Top IMDB movies on Amazon Prime')

In [ ]:
movie_type = prime_movies['mov_type'].value_counts().index.tolist()
print(movie_type[:10])

In [ ]:
def movie_plot(movie) : 
    print(movie)
    thriller_net = prime_movies[prime_movies['mov_type'] == movie]
    net_movies = thriller_net.sort_values(by='IMDb', ascending=False).head(10)
    sns.barplot(data=net_movies, y='Title',x='IMDb')
    plt.title('Top 10 {} movies on Amazon Prime'.format(movie))

In [ ]:
movie_plot('thriller')

In [ ]:
movie_plot('sci-fi')

In [ ]:
movie_plot('action')

In [ ]:
movie_plot('horror')

In [ ]:
movie_plot('animation')

*Hypothesis Testing*

In [ ]:
points=movie_data['IMDb']
mu=points.mean()
sigma=points.std(ddof=0)
print("mu: ", mu, ", sigma:", sigma)

H0: The sample is from the MoviesOnStreamingPlatforms, x_bar = μ.

HA: The sample is not from the MoviesOnStreamingPlatforms, x_bar != (not equal) μ.

In [ ]:
movie_data['IMDb'].head().mean()

The above value is taken as the sample from the population = x_bar = 8.66

In [ ]:
z_critical = 1.96 # alpha level of 0.05 and two-tailed test
x_bar = 8.66
N = 5
SE = sigma/np.sqrt(N)
z_stat = (x_bar - mu)/SE
print(z_stat)

Since z_stat is greater than z_critical we reject the null hypothesis and accept the althernative. Statistically, we say the sample mean is different than the population mean and thus the sample is drawn from the population.

But what if the sample size was larger? Let's redo the calculation with N=300.

In [ ]:
N = 300;
SE = sigma/np.sqrt(N)
z_stat = (x_bar - mu)/SE
print(z_stat)

Here too we get a value of z_stat which is greater than z_critical. Hence we accept Alternate hypothesis.

*Correlation*

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie,axis,show
%matplotlib inline

In [ ]:
movie_data = pd.read_csv("/kaggle/input/movies-on-netflix-prime-video-hulu-and-disney/MoviesOnStreamingPlatforms_updated.csv")
df_corr = movie_data.corr()
fields = ['Type', 'ID']
# drop rows
df_corr.drop(fields, inplace=True)
# drop cols
df_corr.drop(fields, axis=1, inplace=True)

In [ ]:
corr = movie_data.corr(method='kendall')
plt.figure(figsize=(14,8))
sns.heatmap(corr,annot=True)
plt.savefig("correlation.png")
movie_data.columns